# Max 2-Sat

For the problem representation we used a standard format, widely accepted for boolean formulas in CNF, called [DIMACS](https://www.cs.utexas.edu/users/moore/acl2/manuals/current/manual/index-seo.php/SATLINK____DIMACS).

`usage` : change `PATH` variable, in the cell containing parameters, to choose the file of the problem you want to solve



In [10]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from neal import SimulatedAnnealingSampler

import numpy as np
import dwavebinarycsp
import dimod

In [11]:
# parameters

TOKEN = 'DEV-dadab771e1ad528a25146defb4c58b1e0fe50b33' # API dwave
PATH = 'testing_sat/02-sat.txt' # current problem to solve
SAT_PATH = 'testing_sat/'
NUM_READS_SQA = 20
NUM_READS_QPU = 20

# Create QUBO

In order to create the QUBO of a MAX 2 SAT problem we followed [Glover 2019](https://arxiv.org/pdf/1811.11538.pdf). The formulas to convert traditional constraints into quadratic penalties can be found on page 15:


1. No negations: 
    Example $\left(x_{i} \vee x_{j}\right)$ 
    * Traditional constraint: $x_{i}+x_{j} \geq 1$ 
    * Quadratic Penalty: $\left(1-x_{i}-x_{j}+x_{i} x_{j}\right)$
2. One negation:
    Example $\left(x_{i} \vee \bar{x}_{j}\right)$
    * Traditional constraint: $x_{i}+\bar{x}_{j} \geq 1$
    * Quadratic Penalty: $\left(x_{j}-x_{i} x_{j}\right)$
3. Two negations: 
    Example $\left(\bar{x}_{i} \vee \bar{x}_{j}\right)$ 
    * Traditional constraint: $\bar{x}_{i}+\bar{x}_{j} \geq 1$
    * Quadratic Penalty: $\left(x_{i} x_{j}\right)$


In [12]:
# extract clauses and num of variables from 
def extract_clauses(path):
    with open(path, "r") as f:
        # retrieve data from file
        sat = f.readlines()
        data = sat[0].split(" ")

    n_variables = int(data[2]) 

    sat = sat[1:]
    clauses = [x.replace(' 0\n', '') for x in sat]
    
    return clauses, n_variables

clauses, n_variables = extract_clauses(PATH) 
print(f'num_clauses {len(clauses)}')
print('-' * 30)  
print(f'num var {n_variables}')  
print('-' * 30)  
print(f'clauses {clauses}')  

num_clauses 12
------------------------------
num var 4
------------------------------
clauses ['1 2', '1 -2', '-1 2', '-1 -2', '-1 3', '-1 -3', '2 -3', '2 4', '-2 3', '-2 -3', '3 4', '-3 -4']


In [13]:
# generate the matrix Q and c

def gen_q(clauses, n_variables):
    
    Q = np.zeros(shape=[n_variables, n_variables])
    c = 0
    for clause in clauses:
        clause = clause.split(' ')
        int_clause = [int(c) for c in clause]

        s1, s2 = int_clause[0], int_clause[1]
        v1, v2 = abs(s1)-1, abs(s2)-1
        

        if s1 > 0 and s2 > 0: # True True
            # 1 - x1 - x2 + x1x2
            c += 1
            Q[v1][v1] += -1 
            Q[v2][v2] += -1
            Q[v1][v2] += 1/2 
            Q[v2][v1] += 1/2
        elif s1 > 0 and s2 < 0: # True False
            # x2 - x1x2
            Q[v2][v2] += 1
            Q[v1][v2] += -1/2 
            Q[v2][v1] += -1/2

        elif s1 < 0 and s2 > 0: # False True
            # x1 - x1x2
            Q[v1][v1] += 1
            Q[v1][v2] += -1/2 
            Q[v2][v1] += -1/2

        elif s1 < 0 and s2 < 0: # False False
            # x1x2
            Q[v1][v2] += 1/2 
            Q[v2][v1] += 1/2

        else:
            pass # throw error
            
    return Q, c

Q, c = gen_q(clauses, n_var)

print(f'Q is: \n{Q}')
print('-' * 25)
print(f'c is {c}')

Q is: 
[[ 1.   0.   0.   0. ]
 [ 0.   0.  -0.5  0.5]
 [ 0.  -0.5  0.   1. ]
 [ 0.   0.5  1.  -2. ]]
-------------------------
c is 3


## Create BQM

In [14]:
# Create BQM
variable_order = ["x_{}".format(n) for n in range(1, n_variables+1)]
BQM = dimod.BinaryQuadraticModel.from_numpy_matrix(Q, variable_order = variable_order)
BQM

BinaryQuadraticModel({x_1: 1.0, x_2: 0.0, x_3: 0.0, x_4: -2.0}, {('x_2', 'x_3'): -1.0, ('x_2', 'x_4'): 1.0, ('x_3', 'x_4'): 2.0}, 0.0, 'BINARY')

# Solve

We now solve the problem through both simulated and real quantum annealing showing the results obtained.  

In [15]:
def print_response_data(response):
    # ------- Print results to user -------
    print('-' * 160)
    print('{:>40s}{:>40s}{:^40s}{:^40s}'.format('Set 0','Set 1','Energy',"Count"))
    print('-' * 160)
    for sample, E, occ in response.data(fields=['sample','energy',"num_occurrences"]):
        S0 = [k for k,v in sample.items() if v == 0]
        S1 = [k for k,v in sample.items() if v == 1]
        print('{:>40s}{:>40s}{:^40s}{:^40s}'.format(str(S0),str(S1),str(E),str(occ)))

## Simulated Quantum Annealing

In [18]:
def simulated_annealing(bqm, num_reads_sa=NUM_READS_SQA):
    # Run the QUBO on the solver from your config file
    sampler = SimulatedAnnealingSampler()
    response_SQA = sampler.sample(bqm, num_reads=num_reads_sa)
    return response_SQA

## Real Quantum Annealing

In [ ]:
def real_annealing(bqm, num_reads_qpu=NUM_READS_QPU, token=TOKEN):
    # Run the QUBO on the solver from your config file
    sampler = SimulatedAnnealingSampler()
    sampler = EmbeddingComposite(DWaveSampler(token=token))

    response_QPU = sampler.sample(bqm, num_reads=num_reads_qpu)
    return response_QPU

# Results
In the result we select the most frequent solution and we show both the optimal number of unsatisfied constraints and the assignment of the optimal solution

In [40]:
# retrieve result with the best counts

def return_solution(response, Q, c):
    count = 0
    for i in range(len(response)):
        new_count = response[i][2]
        if  new_count > count:
            best = response[i][0]
            count = new_count
    
    y = c + np.matmul(np.matmul(best.T, Q), best) 

    return y, best

def print_solution(response, Q, c):
    y, best = return_solution(response_SQA.aggregate().record, Q, c)
    print(f'y is {int(y)} (optimal number of unsitisfied constraints)')

    for v in range(n_variables):
        print(f'x_{v+1} = {True if best[v] else False}')
    

## Simulated Quantum Annealing

In [44]:
response_SQA = simulated_annealing(BQM)

In [45]:
print_response_data(response_SQA.aggregate())

----------------------------------------------------------------------------------------------------------------------------------------------------------------
                                   Set 0                                   Set 1                 Energy                                  Count                  
----------------------------------------------------------------------------------------------------------------------------------------------------------------
                   ['x_1', 'x_2', 'x_3']                                 ['x_4']                  -2.0                                     20                   


In [43]:
print_solution(response_SQA, Q, c)

y is 1 (optimal number of unsitisfied constraints)
x_1 = False
x_2 = False
x_3 = False
x_4 = True


## Real Quantum Annealing


In [46]:
response_QPU = real_annealing(BQM)

In [47]:
print_response_data(response_QPU)

----------------------------------------------------------------------------------------------------------------------------------------------------------------
                                   Set 0                                   Set 1                 Energy                                  Count                  
----------------------------------------------------------------------------------------------------------------------------------------------------------------
                   ['x_1', 'x_2', 'x_3']                                 ['x_4']                  -2.0                                     20                   


In [49]:
print_solution(response_QPU, Q, c)

y is 1 (optimal number of unsitisfied constraints)
x_1 = False
x_2 = False
x_3 = False
x_4 = True


# Testing

In [ ]:
# define the csp
csp = dwavebinarycsp.factories.random_2in4sat(4, 2) # 8 variables, 4 clauses
csp

In [ ]:


# define the csp
csp = dwavebinarycsp.factories.random_2in4sat(4, 2) 
print(type(csp))
for i in range(len(csp.constraints)):
    print(csp.constraints[i])

# generate the bqm from the csp
import warnings
np.warnings.filterwarnings("ignore", category=DeprecationWarning)

bqm = dwavebinarycsp.stitch(csp)
bqm

<class 'dwavebinarycsp.core.csp.ConstraintSatisfactionProblem'>
Constraint.from_configurations(frozenset({(0, 0, 1, 0), (0, 1, 1, 1), (1, 0, 1, 1), (0, 1, 0, 0), (1, 0, 0, 0), (1, 1, 0, 1)}), (0, 1, 2, 3), Vartype.BINARY, name='2-in-4')
Constraint.from_configurations(frozenset({(0, 0, 1, 0), (0, 1, 1, 1), (1, 0, 1, 1), (0, 1, 0, 0), (1, 0, 0, 0), (1, 1, 0, 1)}), (0, 1, 3, 2), Vartype.BINARY, name='2-in-4')


BinaryQuadraticModel({0: -4.0, 1: -4.0, 2: 4.0, 3: 4.0}, {(0, 1): 8.0, (0, 2): 0.0, (0, 3): 0.0, (1, 2): 0.0, (1, 3): 0.0, (2, 3): -8.0}, 4.0, 'BINARY')

In [ ]:
resp_san = simulated_annealing(bqm)
resp_qpu = real_annealing(bqm)

print_response_data(resp_san.aggregate())
print_response_data(resp_qpu.aggregate())


----------------------------------------------------------------------------------------------------------------------------------------------------------------
                                   Set 0                                   Set 1                 Energy                                  Count                  
----------------------------------------------------------------------------------------------------------------------------------------------------------------
                               [0, 2, 3]                                     [1]                  0.0                                      4                    
                               [0, 1, 2]                                     [3]                  0.0                                      5                    
                                     [3]                               [0, 1, 2]                  0.0                                      7                    
                                  

In [ ]:
Q_sim, y_sim = return_solution()